<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
     <h3> Objective</h3> <br>
    <b style = "color:#000000"><i>Attach a new Policy to an IAM User</i></b>
</div>
<br>
</center>

<center><h2><u>Update and Manage AWS User Permissions</u></h2></center>

# Steps Overview
1)[ List all attached policies](#1)<br>
2)[ Attach new policy](#2)<br>

<h3><a id='1'>List all attached policies</a></h3>
Using unSkript's AWS List Attached User Policies action we will list all policies for a given user.

>Action takes the following parameters: `aws_username`

In [29]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from botocore.exceptions import ClientError
import pprint


from beartype import beartype
@beartype
def aws_list_attached_user_policies_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_attached_user_policies(handle, user_name: str) -> List:
    """aws_list_attached_user_policies returns the list of policies attached to the user.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type user_name: string
        :param user_name: IAM user whose policies need to fetched.

        :rtype: List with with the attched policy names.
    """
    result = []
    ec2Client = handle.client('iam')
    try:
        response = ec2Client.list_attached_user_policies(UserName=user_name)
        for i in response["AttachedPolicies"]:
            result.append(i['PolicyName'])

    except ClientError as error:
        result.append(error.response)

    return result


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "user_name": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "aws_username",
    "iter_parameter": "user_name"
    }''')
task.configure(outputName="all_policies")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_attached_user_policies, lego_printer=aws_list_attached_user_policies_printer, hdl=hdl, args=args)

<h3>Check if policy already exists</h3>
Using the ouput from Step 1,this action filters out users which already have the given policy added and returns users which don't.

In [27]:
def get_policies():
    r_list=[]
    for k,v in all_policies.items():
        r = {}
        if policy_name not in v:
            r_list.append(k)
    return r_list

user_policy_list = get_policies()

task.configure(outputName="user_policy_list")

<h3><a id='2'>Attach New Policy </a></h3>
Using unSkript's AWS Attach IAM Policy Lego action we will attach the given policiy to user.

>Action takes the following parameters: `aws_username`, `policy_name`

In [28]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from botocore.exceptions import ClientError
import pprint


from beartype import beartype
@beartype
def aws_attach_iam_policy_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_attach_iam_policy(handle, user_name: str, policy_name: str) -> Dict:
    """aws_attache_iam_policy used to provide user permissions.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type user_name: string
        :param user_name: Dictionary of credentials info.

        :type policy_name: string
        :param policy_name: Policy name to apply the permissions to the user.

        :rtype: Dict with User policy information.
    """
    result = {}
    iamResource = handle.resource('iam')
    try:
        user = iamResource.User(user_name)
        response = user.attach_policy(
            PolicyArn='arn:aws:iam::aws:policy/'+policy_name
            )
        result = response
    except ClientError as error:
        result = error.response

    return result


def unskript_default_printer(output):
    if isinstance(output, (list, tuple)):
        for item in output:
            print(f'item: {item}')
    elif isinstance(output, dict):
        for item in output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(output)

task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "policy_name": "policy_name",
    "user_name": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "user_policy_list",
    "iter_parameter": "user_name"
    }''')
task.configure(outputName="final_result")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_attach_iam_policy, lego_printer=unskript_default_printer, hdl=hdl, args=args)

<p>In this Runbook, we were able to list all the attached policies to IAM user and and attach a new policy to the same user using unSkript's AWS actions. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io">us.app.unskript.io</a></p>